In [19]:
import os

In [20]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MLOPS_project'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MLOPS_project'

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [22]:
from MLOPS_project.constants import *
from MLOPS_project.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [24]:
import os
from MLOPS_project import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [25]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        data['type'] = data['type'].map({'red': 1, 'white': 0})

        for col in ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "pH", "sulphates"]:
            data[col].fillna(data[col].mean(), inplace=True)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [26]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-07-05 16:10:16,995: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-05 16:10:16,997: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-05 16:10:17,005: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-05 16:10:17,005: INFO: common: created directory at: artifacts]
[2025-07-05 16:10:17,005: INFO: common: created directory at: artifacts/data_transformation]
[2025-07-05 16:10:17,088: INFO: 504488821: Splited data into training and test sets]
[2025-07-05 16:10:17,088: INFO: 504488821: (4872, 13)]
[2025-07-05 16:10:17,088: INFO: 504488821: (1625, 13)]
(4872, 13)
(1625, 13)


C:\Users\Dheer\AppData\Local\Temp\ipykernel_28772\504488821.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
